**Note**: microTVM is no longer in active development and has been phased out since [0.18.0](https://discuss.tvm.apache.org/t/phasing-out-legacy-components/17703/23)

In [ ]:
!python -m tf2onnx.convert --tflite apache-tvm/cifar10.tflite --output apache-tvm/cifar10.onnx

In [9]:
import onnx
onnx_file = onnx.load("cifar10.onnx")

Convert the TensorFlow Lite model into a relay module object (mod). The relay module is what we will provide to the TVM compiler along with the target device and compiler options, to generate the code.

In [32]:
from tvm.relax.frontend.onnx import from_onnx
mod = from_onnx(onnx_file, keep_params_in_input=False)

OpNotImplemented: The following operators are not supported for frontend ONNX: DequantizeLinear, QuantizeLinear

**OpNotImplemented**: The following operators are not supported for frontend ONNX: DequantizeLinear, QuantizeLinear

Define the host machine as the target device.

In [31]:
import tvm
#target = tvm.target.Target.from_device('cpu')  # Host machine
target = tvm.target.arm_cpu(model="cortex-m4", options=["--mtriple=arm-none-eabi", "--mcpu=cortex-m4", "--mattr=+fpv4-sp-d16"])
print(target)

llvm -keys=arm_cpu,cpu -device=arm_cpu -mattr=+fpv4-sp-d16 -mcpu=cortex-m4 -model=cortex-m4 -mtriple=arm-none-eabi


In [ ]:
import numpy as np
mod = relax.get_pipeline("zero")(mod)
executable = tvm.compile(mod, target)

In [ ]:
ARTIFACT_DIR = Path("relax_export_artifacts")
ARTIFACT_DIR.mkdir(exist_ok=True)


library_path = ARTIFACT_DIR / "cifar10.so"
executable.export_library(str(library_path), workspace_dir=str(ARTIFACT_DIR))